# 라이브러리 설치

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# !apt-get update << 리눅스 os를 업데이트하는 것(생략 가능)
!apt-get install g++ openjdk-8-jdk    # Java 설치

# JPype 설치
!pip install JPype1

# RHINO 설치
!pip install rhinoMorph

# 한국 시간 추출을 위한 pytz 패키지 설치
!pip install pytz

# 인터페이스 gradio 패키지 설치
!pip install gradio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubuntu1).
g++ set to manually installed.
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libfontenc1 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common libice-dev librsvg2-common libsm-dev
  libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-8-jdk-headless
  openjdk-8-jre openjdk-8-jre-headless x11-utils
Suggested packages:
  gvfs libice-doc libsm-doc libxt-doc openjdk-8-demo openjdk-8-source visualvm
  libnss-mdns fonts-nanum fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei fonts-wqy-zenhei fonts-indic mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libfontenc1 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common l

In [ ]:
import rhinoMorph
import pandas as pd
import difflib
import joblib
import time
from datetime import datetime, timedelta
import pytz
import re
import gradio as gr



In [ ]:
# RhinoMorph 초기화
rn = rhinoMorph.startRhino()

# 아이템 이름 키워드
itemname = {
  'item': ['견고한 수호자의 모래시계', '무결점 골든 베릴', '무결점 라이언 코어', '무결점 조화의 결정체', '백룡의 뿔', '균열의 단편', '수호자의 모래시계',
           '왜곡된 차원의 큐브[교환가능]', '차오르는 수호자의 망토', '차오르는 수호자의 모래시계', '초전도 에너지 코어 상자[1회 교환가능]', '축제의 여왕 페리아 알', '페스타의 추억 칭호 상자']}



filepath:  /usr/local/lib/python3.10/dist-packages
classpath:  /usr/local/lib/python3.10/dist-packages/rhinoMorph/lib/rhino.jar
RHINO started!


In [ ]:
# [함수] 입력문의 형태소를 분석하여 리스트 형태로 반환
def Morphology_Analysis(message):
  result= rhinoMorph.onlyMorph_list(rn, message, pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ', 'SN'])
  return result

# [함수] 입력문에서 패턴 1, 2와 일치하는 시간 키워드 추출하여 현재와 예측 시간의 차이를 반환(시간 단위)
def time(message):
  # 현재 한국 표준시간대의 현재 날짜 및 시간 정보 가져오기
  korea_timezone = pytz.timezone('Asia/Seoul')
  now = datetime.now(korea_timezone)

  # 패턴 1 - ex) 8월 17일 15시
  pattern = r'(\d+)월\s*(\d+)일\s*(\d+)시'
  match = re.search(pattern, message)
  time_result=''
  if match is not None:
    target_month = int(match.group(1))
    target_day = int(match.group(2))
    target_hour = int(match.group(3))

    # 예측 시간
    target_time = datetime(now.year,target_month, target_day, target_hour)
    # 현재 시간
    current_time = datetime(now.year,now.month, now.day, now.hour)

    # 두 시간의 차이 계산
    time_difference = (target_time - current_time).total_seconds() // 3600
    time_result = int(time_difference)

  # 패턴 2 - ex) 8월 17일 15시
  else:
    pattern2 = r'(\d+)시간\s*(뒤|후)'      # 사용자 입력문에서 추출할 시간 형태
    match2 = re.search(pattern2, message)
    if match2 is not None:
      time_result = int(match2.group(1))
  return time_result

def run(message):
  # 현재 한국 표준시간대의 현재 날짜 및 시간 정보 가져오기
  korea_timezone = pytz.timezone('Asia/Seoul')
  now = datetime.now(korea_timezone)
  time_result = time(message)
  nouns = Morphology_Analysis(message)
  keywords = itemname['item']   # 비교대상이 되는 아이템 키워드

  input_items = []
  # 입력받은 text 에 있는 아이템 이름들 중 일치하는 것 전부 추출
  for item in keywords:
    if (message).find(item) != -1:
      input_items.append(item)
  if input_items:
    similar_keyword = input_items
  else:
    similar_keyword = difflib.get_close_matches(' '.join(nouns), keywords, n=5, cutoff = 0.3)

  # 검색된 아이템 : 0
  if not similar_keyword:
    bot_message = '검색된 아이템이 없습니다. 아이템 이름과 정확히 입력해 주세요.'
  elif type(time_result) == str:
    bot_message = '아이템 이름과 시간을 정확히 입력해 주세요.'

  # 검색된 아이템 : 1
  elif len(similar_keyword)==1:
    # 모델 불러오기
    path = '/content/gdrive/MyDrive/[광인사] 1차 프로젝트/코드정리/'
    model = joblib.load(path + f'graph/{similar_keyword[0]}/' + f'Huber_{similar_keyword[0]}.pkl')

    data = pd.read_csv(path+f'preprocessing_file/mentoring/{similar_keyword[0]}_merged_preprocessing.csv')
    df = pd.read_csv(path+f'merged_file/{similar_keyword[0]}_merged.csv')
    IQR = df['unitPrice'].quantile(.75) - df['unitPrice'].quantile(.25)
    max_range = df['unitPrice'].quantile(.75) + 1.5 * IQR
    min_range = df['unitPrice'].quantile(.25) - 1.5 * IQR

    data["soldDate"] = pd.to_datetime(data["soldDate"])
    sold_month = data["soldDate"].dt.month
    sold_day = data["soldDate"].dt.day
    sold_hour = data["soldDate"].dt.hour
    X_test = pd.DataFrame({'month': sold_month, 'day':sold_day, 'hour':sold_hour, 'unitPrice_past_0': data["unitPrice_past_0"]})
    length = len(X_test)
    pre_price = model.predict(X_test.iloc[length-1:length])[0]

    for i in range(time_result):
      i_time = datetime.now(korea_timezone)+timedelta(hours=i)
      i_time_month = i_time.month
      i_time_day = i_time.day
      i_time_hour = i_time.hour
      a = pd.DataFrame({'month': [i_time_month], 'day': [i_time_day], 'hour': [i_time_hour], 'unitPrice_past_0': [pre_price]})
      pre_price = model.predict(a)[0]

    if (pre_price>max_range)|(pre_price<min_range):
      bot_message = f'{similar_keyword[0]}의 {i_time_month}월 {i_time_day}일 {i_time_hour+1}시 예상 가격은 {"{:,}".format(int(pre_price))}골드입니다.\n이상 거래가 감지되었으니 주의바랍니다.'

    else:
      bot_message = f'{similar_keyword[0]}의 {i_time_month}월 {i_time_day}일 {i_time_hour+1}시 예상 가격은 {"{:,}".format(int(pre_price))}골드입니다.'

  # 검색된 아이템 : 2개 이상
  else:
    bot_message = f'혹시 아래의 아이템 중 하나를 검색하셨나요? 더 정확히 입력해주세요.\n'
    for i in range(len(similar_keyword)):
      bot_message +=  f'\n{i+1}번, {similar_keyword[i]}'
  return bot_message

def abs(message,history):
  bot_message = run(message)
  return bot_message

demo = gr.ChatInterface(
    fn = abs,
    textbox=gr.Textbox(placeholder="어떤 아이템의 가격을 알고 싶으신가요?", container=False, scale=15),
    title="모코코",
    description="가격을 예측하고 싶은 아이템 이름과 날짜를 포함하여 입력해주세요. 날짜 예시) 1시간 뒤, 8월 15일 17시",
    retry_btn="다시 보내기",
    undo_btn="이전챗 삭제",
    clear_btn="전챗 삭제",)

demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:

similar_keyword = difflib.get_close_matches(' '.join(nouns), keywords, n=5, cutoff = 0.3)

In [ ]:
################################# 철리 수정용 ################################################

In [ ]:
import copy
# [함수] 입력문의 형태소를 분석하여 리스트 형태로 반환
def Morphology_Analysis(message):
  result= rhinoMorph.onlyMorph_list(rn, message, pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ', 'SN'])
  return result

# [함수] 입력문에서 패턴 1, 2와 일치하는 시간 키워드 추출하여 현재와 예측 시간의 차이를 반환(시간 단위)
def time(message):
  # 현재 한국 표준시간대의 현재 날짜 및 시간 정보 가져오기
  korea_timezone = pytz.timezone('Asia/Seoul')
  now = datetime.now(korea_timezone)

  # 패턴 1 - ex) 8월 17일 15시
  pattern = r'(\d+)월\s*(\d+)일\s*(\d+)시'
  match = re.search(pattern, message)
  time_result=''
  if match is not None:
    target_month = int(match.group(1))
    target_day = int(match.group(2))
    target_hour = int(match.group(3))

    # 예측 시간
    target_time = datetime(now.year,target_month, target_day, target_hour)
    # 현재 시간
    current_time = datetime(now.year,now.month, now.day, now.hour)

    # 두 시간의 차이 계산
    time_difference = (target_time - current_time).total_seconds() // 3600
    time_result = int(time_difference)

  # 패턴 2 - ex) 8월 17일 15시
  else:
    pattern2 = r'(\d+)시간\s*(뒤|후)'      # 사용자 입력문에서 추출할 시간 형태

    match2 = re.search(pattern2, message)
    if match2 is not None:
      time_result = int(match2.group(1))
  return time_result

def run(message):
  time_result = time(message)
  print(time_result)
  nouns = Morphology_Analysis(message)
  keywords = itemname['item']   # 비교대상이 되는 아이템 키워드
  # 현재 한국 표준시간대의 현재 날짜 및 시간 정보 가져오기
  korea_timezone = pytz.timezone('Asia/Seoul')
  now = datetime.now(korea_timezone)

  input_items = []
  # 입력받은 text 에 있는 아이템 이름들 중 일치하는 것 전부 추출
  for item in keywords:
    if (message).find(item) != -1 or (message).find(''.join(item.split())) != -1:
      input_items.append(item)
    if len(input_items)>1:
      name = ''
      for item_name in input_items:
        if len(item_name) > len(name):
          name = item_name
      input_items = [name]
  if input_items:
    similar_keyword = input_items
  else:
    print('이전 nouns', nouns)
    pattern = r'(\d+)월\s*(\d+)일\s*(\d+)시'
    match = re.search(pattern, message)
    if match is not None:
      target_month = match.group(1)
      target_day = match.group(2)
      target_hour = match.group(3)
      if target_month in nouns: nouns.remove(target_month)
      if target_day in nouns: nouns.remove(target_day)
      if target_hour in nouns: nouns.remove(target_hour)
      if '가격' in nouns: nouns.remove('가격')
      if '알리' in nouns: nouns.remove('알리')
      if '시간' in nouns: nouns.remove('시간')

    pattern2 = r'(\d+)시간\s*(뒤|후)'      # 사용자 입력문에서 추출할 시간 형태
    match2 = re.search(pattern2, message)
    if match2 is not None:
      target_hour = match2.group(1)
      if target_hour in nouns: nouns.remove(target_hour)
      if '가격' in nouns: nouns.remove('가격')
      if '알리' in nouns: nouns.remove('알리')
      if '시간' in nouns: nouns.remove('시간')
      if '뒤' in nouns: nouns.remove('뒤')
      if '후' in nouns: nouns.remove('후')

    print('nouns', nouns)
    print('keywords', keywords)
    similar_keyword = difflib.get_close_matches(' '.join(nouns), keywords, n=5, cutoff = 0.3)


  # 검색된 아이템 : 0
  if not similar_keyword:
    bot_message = '검색된 아이템이 없습니다. 아이템 이름과 정확히 입력해 주세요.'
  elif type(time_result) == str:
    bot_message = '아이템 이름과 시간을 정확히 입력해 주세요.'

  # 검색된 아이템 : 1
  elif len(similar_keyword)==1:
    # 모델 불러오기
    path = '/content/gdrive/MyDrive/[광인사] 1차 프로젝트/코드정리/'
    model = joblib.load(path + f'graph/{similar_keyword[0]}/' + f'Huber_{similar_keyword[0]}.pkl')

    data = pd.read_csv(path+f'preprocessing_file/mentoring/{similar_keyword[0]}_merged_preprocessing.csv')
    data["soldDate"] = pd.to_datetime(data["soldDate"])
    sold_month = data["soldDate"].dt.month
    sold_day = data["soldDate"].dt.day
    sold_hour = data["soldDate"].dt.hour
    X_test = pd.DataFrame({'month': sold_month, 'day':sold_day, 'hour':sold_hour, 'unitPrice_past_0': data["unitPrice_past_0"]})
    length = len(X_test)
    pre_price = model.predict(X_test.iloc[length-1:length])[0]

    for i in range(time_result):
      i_time = datetime.now(korea_timezone)+timedelta(hours=i)
      i_time_month = i_time.month
      i_time_day = i_time.day
      i_time_hour = i_time.hour
      a = pd.DataFrame({'month': [i_time_month], 'day': [i_time_day], 'hour': [i_time_hour], 'unitPrice_past_0': [pre_price]})
      pre_price = model.predict(a)[0]

    bot_message = f'{similar_keyword[0]}의 {i_time_month}월 {i_time_day}일 {i_time_hour+1}시 예상 가격은 {"{:,}".format(int(pre_price))}골드입니다.'

  # 검색된 아이템 : 2개 이상
  else:
    bot_message = f'혹시 아래의 아이템 중 하나를 검색하셨나요? 더 정확히 입력해주세요.\n'
    for i in range(len(similar_keyword)):
      bot_message +=  f'\n{i+1}번, {similar_keyword[i]}'
  return bot_message
def abs(message,history):
  bot_message = run(message)
  return bot_message

demo = gr.ChatInterface(
    fn = abs,
    textbox=gr.Textbox(placeholder="어떤 아이템의 가격을 알고 싶으신가요?", container=False, scale=15),
    title="모코코",
    description="가격을 예측하고 싶은 아이템 이름과 날짜를 포함하여 입력해주세요. 날짜 예시) 1시간 뒤, 8월 15일 17시",
    retry_btn="다시 보내기",
    undo_btn="이전챗 삭제",
    clear_btn="전챗 삭제",)

demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
run('차오르는 수호자 뭐시기 8월 16일 20시 가격을 알려줘')

2
이전 nouns ['차오르', '수호자', '8', '16', '20', '가격', '알리']
nouns ['차오르', '수호자']
keywords ['견고한 수호자의 모래시계', '무결점 골든 베릴', '무결점 라이언 코어', '무결점 조화의 결정체', '백룡의 뿔', '균열의 단편', '수호자의 모래시계', '왜곡된 차원의 큐브[교환가능]', '차오르는 수호자의 망토', '차오르는 수호자의 모래시계', '초전도 에너지 코어 상자[1회 교환가능]', '축제의 여왕 페리아 알', '페스타의 추억 칭호 상자']


'혹시 아래의 아이템 중 하나를 검색하셨나요? 더 정확히 입력해주세요.\n\n1번, 차오르는 수호자의 망토\n2번, 차오르는 수호자의 모래시계\n3번, 견고한 수호자의 모래시계\n4번, 수호자의 모래시계\n5번, 페스타의 추억 칭호 상자'

In [ ]:
rhinoMorph.onlyMorph_list(rn, '백용의 뿔', pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ', 'SN'])

['백', '뿔']

In [ ]:
rhinoMorph.onlyMorph_list(rn, '차오르는 수호자의 모래시계', pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ', 'SN'])

['차오르', '수호자', '모래시계']

In [ ]:
print('사용자 입력문 :', Morphology_Analysis('균열의 단편 2시간 뒤의 가격을 알려줘'))
print('아이템 이름 :', rhinoMorph.onlyMorph_list(rn, '균열의 단편', pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ', 'SN']))
print('='*70)
print('사용자 입력문 :', Morphology_Analysis('무결점 라이언 코어의 8월 17일 14시 가격을 알려줘'))
print('아이템 이름 :', rhinoMorph.onlyMorph_list(rn,'무결점 라이언 코어',pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ', 'SN']))


사용자 입력문 : ['균열', '단편', '2', '시간', '뒤', '가격', '알리']
아이템 이름 : ['균열', '단편']
사용자 입력문 : ['결점', '라이언', '코어', '8', '17', '14', '가격', '알리']
아이템 이름 : ['결점', '라이언', '코어']
